## 1. セットアップ

In [ ]:
# 必要なライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
import sys
import os
from datetime import datetime

warnings.filterwarnings('ignore')
from pathlib import Path

rootPath = Path.cwd().parent
sys.path.append(str(rootPath))
from src.timeseries_processing import SequenceCreator, DataSplitter, DataStandardizer, feature_groups_route_based

feature_groups = feature_groups_route_based

# 日本語フォント設定（可視化用）
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style('whitegrid')

print(f"TensorFlow version: {tf.__version__}")
print(f"Analysis started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

data_path = f'{rootPath}/data/delay_analysis_route_based.csv'
print(f"Loading data from {data_path}...")
model_path = f'{rootPath}/files/model/best_delay_model_route_based_20251102_061332.h5'
print(f"Loading model from {model_path}...")
# ConvLSTM用の特徴量グループ定義

## 2. データ読み込みと前処理

In [ ]:
# データ読み込み
print("Loading dataset...")
delay_features = pd.read_csv(data_path)

print(f"Dataset shape: {delay_features.shape}")
print(f"\nColumn names ({len(delay_features.columns)} features):")
print(delay_features.columns.tolist())

# 基本統計
print(f"\nBasic statistics:")
print(f"- Routes: {delay_features['route_id'].nunique()}")
print(f"- Date range: {delay_features['time_bucket'].min()} to {delay_features['time_bucket'].max()}")
print(f"- Mean delay: {delay_features['arrival_delay'].mean():.2f} seconds")

## 4. 時系列データの作成

In [ ]:
# 時系列シーケンス作成
print("Creating time series sequences...")
print("Note: feature_groups has intentional duplicate (distance_from_downtown_km appears twice)")

sequence_creator = SequenceCreator(
    input_timesteps=8, 
    output_timesteps=3,
    feature_groups=feature_groups
)

X_delay, y_delay, route_direction_info, used_features, feature_group_info = sequence_creator.create_route_direction_aware_sequences(
    delay_features,
    spatial_organization=True
)

print(f"\nSequence shapes:")
print(f"  X: {X_delay.shape}")
print(f"  y: {y_delay.shape}")
print(f"\nUsed features ({len(used_features)}):")
for i, feat in enumerate(used_features, 1):
    print(f"  {i}. {feat}")

if feature_group_info:
    print(f"\n=== Feature Group Info ===")
    for group_name, info in feature_group_info.items():
        print(f"{group_name}: {info['features']} (indices {info['start_idx']}:{info['end_idx']})")

## 5. データ分割

In [ ]:
# データ分割
print("Splitting data...")
splitter = DataSplitter()

X_train, X_test, y_train, y_test, train_routes, test_routes = splitter.train_test_split_by_route_direction(
    X_delay, y_delay, route_direction_info, train_ratio=0.9
)
standardizer = DataStandardizer()
X_train_scaled = standardizer.fit_transform_features(X_train)
X_test_scaled = standardizer.transform_features(X_test)

actual_feature_count = X_train.shape[2]
X_train_reshaped = splitter.reshape_for_convlstm(
    X_train_scaled, target_height=1, target_width=actual_feature_count
)
X_test_reshaped = splitter.reshape_for_convlstm(
    X_test_scaled, target_height=1, target_width=actual_feature_count
) 
print(f"\nData split:")
print(f"  Train: X={X_train_reshaped.shape}, y={y_train.shape}")
print(f"  Test:  X={X_test_reshaped.shape}, y={y_test.shape}")

## 6. モデル読み込み

学習済みのConvLSTMモデルを読み込みます。

In [ ]:
# モデル読み込み
print(f"Loading model from {model_path}...")
model = tf.keras.models.load_model(model_path, compile=False)

# モデル情報表示
print("\nModel loaded successfully!")
print(f"Input shape: {model.input.shape}")
print(f"  → Expected: (batch, 8 timesteps, 1 height, 17 width/features, 1 channels)")
print(f"Output shape: {model.output.shape}")
print(f"Total parameters: {model.count_params():,}")

# データ形状が一致するか確認
expected_width = model.input.shape[3]
actual_width = X_train_reshaped.shape[3]
print(f"\n=== Shape Compatibility Check ===")
print(f"Model expects: width={expected_width} features")
print(f"Data provides: width={actual_width} features")

if expected_width != actual_width:
    raise ValueError(
        f"Shape mismatch! Model expects {expected_width} features, "
        f"but data has {actual_width} features. "
        f"Check feature_groups definition."
    )

print("✓ Shape compatibility confirmed!")

# 基本性能評価
print("\nEvaluating baseline performance...")
y_pred = model.predict(X_test_reshaped, verbose=0)

# 最後のタイムステップで評価
y_test_last = y_test[:, -1] if len(y_test.shape) > 1 else y_test
y_pred_last = y_pred[:, -1] if len(y_pred.shape) > 1 else y_pred

baseline_mae = mean_absolute_error(y_test_last, y_pred_last)
baseline_rmse = np.sqrt(mean_squared_error(y_test_last, y_pred_last))
baseline_r2 = r2_score(y_test_last, y_pred_last)

print(f"\nBaseline Performance (last timestep):")
print(f"  MAE:  {baseline_mae:.4f} seconds ({baseline_mae/60:.2f} minutes)")
print(f"  RMSE: {baseline_rmse:.4f} seconds ({baseline_rmse/60:.2f} minutes)")
print(f"  R²:   {baseline_r2:.4f}")

## 7. カスタムPermutation Importanceの実装

**重要**: 既存の学習済みモデルは入力形状が固定されているため、特徴量を削除/追加すると使用できません。
そのため、以下の戦略を採用します：

### 戦略
1. **4次元入力を保持**: ConvLSTMの入力形状 (samples, timesteps, height, width) をそのまま使用
2. **特徴量レベルでシャッフル**: width次元内の各特徴量インデックスをシャッフル
3. **全タイムステップに適用**: 時系列の連続性を維持しながら特徴量の寄与を測定

### 制限事項
- モデル構造の変更は不可（再学習が必要）
- 特徴量の追加/削除は不可（再学習が必要）
- 特徴量の重要度のみを分析可能

In [ ]:
def custom_permutation_importance_convlstm(model, X, y, feature_group_info, 
                                           n_repeats=10, random_state=42,
                                           shuffle_strategy='cross_sample'):
    """
    ConvLSTM用のカスタムPermutation Importance（時系列対応版・堅牢化）
    
    Args:
        model: 学習済みConvLSTMモデル
        X: 4次元入力 (samples, timesteps, height, width)
        y: 目標値 (samples, output_timesteps)
        feature_group_info: 特徴量グループ情報
        n_repeats: シャッフル回数
        random_state: 乱数シード
        shuffle_strategy: シャッフル戦略
            - 'cross_sample': サンプル間でシャッフル（時系列順序は保持）【推奨】
            - 'block': ブロック単位でシャッフル（局所性を保持）
            - 'within_sample': サンプル内でシャッフル（時系列破壊・非推奨）
    
    Returns:
        importance_dict: {feature_name: {'mean': float, 'std': float, 'raw_scores': list}}
        baseline_mae: ベースラインのMAE
    """
    np.random.seed(random_state)
    
    # ベースライン性能を計算
    y_pred_baseline = model.predict(X, verbose=0)
    y_true_last = y[:, -1] if len(y.shape) > 1 else y
    y_pred_last = y_pred_baseline[:, -1] if len(y_pred_baseline.shape) > 1 else y_pred_baseline
    baseline_mae = mean_absolute_error(y_true_last, y_pred_last)
    
    print(f"Baseline MAE: {baseline_mae:.4f} seconds")
    print(f"Shuffle strategy: {shuffle_strategy}")
    print(f"Sample size: {X.shape[0]}")
    print(f"\nCalculating feature importance...")
    
    # 特徴量名とインデックスのマッピング
    feature_indices = {}
    for group_name, info in feature_group_info.items():
        for i, feat in enumerate(info['features']):
            if feat != 'arrival_delay':  # ターゲットは除外
                width_idx = info['start_idx'] + i
                feature_indices[feat] = width_idx
    
    importance_scores = {}
    
    # 各特徴量について
    for feat_name, width_idx in feature_indices.items():
        print(f"  Processing: {feat_name} (index {width_idx})...")
        mae_increases = []
        
        # n_repeats回シャッフル
        for repeat in range(n_repeats):
            # データのコピー
            X_shuffled = X.copy()
            
            if shuffle_strategy == 'cross_sample':
                # 【推奨】サンプル間でシャッフル（時系列の順序は保持）
                # 例: サンプルAの特徴量 → サンプルBの時系列に配置
                shuffle_idx = np.random.permutation(X.shape[0])
                # 時系列全体をまとめてシャッフル（timestep次元はそのまま）
                X_shuffled[:, :, :, width_idx] = X[shuffle_idx, :, :, width_idx]
                
            elif shuffle_strategy == 'block':
                # ブロック単位でシャッフル（時系列の局所性を保持）
                block_size = 2  # 2タイムステップを1ブロック
                n_timesteps = X.shape[1]
                
                for sample_idx in range(X.shape[0]):
                    # ブロック数
                    n_blocks = n_timesteps // block_size
                    block_perm = np.random.permutation(n_blocks)
                    
                    X_temp = X_shuffled[sample_idx, :, :, width_idx].copy()
                    for new_pos, old_pos in enumerate(block_perm):
                        start_new = new_pos * block_size
                        end_new = min(start_new + block_size, n_timesteps)
                        start_old = old_pos * block_size
                        end_old = min(start_old + block_size, n_timesteps)
                        X_shuffled[sample_idx, start_new:end_new, :, width_idx] = X_temp[start_old:end_old, :]
                        
            elif shuffle_strategy == 'within_sample':
                # サンプル内でシャッフル（時系列破壊・非推奨）
                for sample_idx in range(X.shape[0]):
                    timestep_perm = np.random.permutation(X.shape[1])
                    X_shuffled[sample_idx, :, :, width_idx] = X_shuffled[sample_idx, timestep_perm, :, width_idx]
            
            # シャッフル後の予測
            y_pred_shuffled = model.predict(X_shuffled, verbose=0)
            y_pred_shuffled_last = y_pred_shuffled[:, -1] if len(y_pred_shuffled.shape) > 1 else y_pred_shuffled
            
            # MAE増加量を計算
            shuffled_mae = mean_absolute_error(y_true_last, y_pred_shuffled_last)
            mae_increase = shuffled_mae - baseline_mae
            mae_increases.append(mae_increase)
        
        # 統計量を保存
        importance_scores[feat_name] = {
            'mean': np.mean(mae_increases),
            'std': np.std(mae_increases),
            'raw_scores': mae_increases
        }
    
    return importance_scores, baseline_mae

# 【修正1】サンプル数を大幅に増やす（500 → 5000）
n_samples = min(5000, len(X_train_reshaped))  # 5000サンプルで統計的に安定
X_sample = X_train_reshaped[:n_samples].copy()
y_sample = y_train[:n_samples].copy()  # 【修正2】y_testではなくy_trainを使用

print(f"Using {n_samples} samples for permutation importance")
print(f"Data shape: X={X_sample.shape}, y={y_sample.shape}")
print(f"Data source: X_train[:n_samples] and y_train[:n_samples]")

# 【修正3】cross_sample戦略で実行（時系列の順序を保持）
importance_scores, baseline_mae = custom_permutation_importance_convlstm(
    model, X_sample, y_sample, feature_group_info, 
    n_repeats=10, 
    random_state=42,
    shuffle_strategy='cross_sample'  # 時系列順序を保持しつつサンプル間でシャッフル
)

print("\nPermutation importance calculation completed!")


## 8. 特徴量重要度の集計と分析

In [ ]:
# 重要度データフレーム作成
importance_data = []
for feat_name, scores in importance_scores.items():
    importance_data.append({
        'feature': feat_name,
        'importance_mean': scores['mean'],
        'importance_std': scores['std']
    })

importance_df = pd.DataFrame(importance_data)

# カテゴリを追加
def assign_category(feature_name):
    for category, features in feature_groups.items():
        if feature_name in features:
            return category
    return 'other'

importance_df['category'] = importance_df['feature'].apply(assign_category)

# 重要度でソート
importance_df = importance_df.sort_values('importance_mean', ascending=False)

print("Feature Importance (Top 20):")
print(importance_df.head(20).to_string(index=False))

# カテゴリ別の統計
print("\n" + "="*60)
print("Category-wise Importance Summary:")
print("="*60)
category_stats = importance_df.groupby('category').agg({
    'importance_mean': ['sum', 'mean', 'count']
}).round(4)
category_stats.columns = ['total_importance', 'avg_importance', 'n_features']
category_stats = category_stats.sort_values('total_importance', ascending=False)
print(category_stats)

## 10. 結果の保存

In [ ]:
# 10.1 特徴量重要度の詳細をCSV保存
output_dir = f'{rootPath}/notebook'
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# 詳細CSV
csv_path = os.path.join(output_dir, f'feature_importance_detailed_{timestamp}.csv')
importance_df.to_csv(csv_path, index=False)
print(f"Detailed importance saved to: {csv_path}")

# カテゴリ別サマリーCSV
category_csv_path = os.path.join(output_dir, f'category_importance_summary_{timestamp}.csv')
category_stats.to_csv(category_csv_path)
print(f"Category summary saved to: {category_csv_path}")

## 11. 分析サマリーレポート

In [ ]:
# レポート生成
report = f"""
# 特徴量重要度分析レポート

**分析日時**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
**モデル**: Bidirectional ConvLSTM {model_path.split('/')[-1]}
**データセット**: {data_path.split('/')[-1]}
**手法**: Permutation Importance (n_repeats=10)
**評価指標**: MAE (Mean Absolute Error)

---

## 1. 基本情報

- **総特徴量数**: {len(importance_df)}
- **分析サンプル数**: {n_samples}
- **ベースラインMAE**: {baseline_mae:.4f} 秒 ({baseline_mae/60:.2f} 分)
- **ベースラインRMSE**: {baseline_rmse:.4f} 秒 ({baseline_rmse/60:.2f} 分)
- **ベースラインR²**: {baseline_r2:.4f}

---

## 2. 特徴量ランキング

| Rank | Feature | Category | Importance | Std |
|------|---------|----------|------------|----- |
"""

for idx, row in importance_df.iterrows():
    report += f"| {idx+1} | {row['feature']} | {row['category']} | {row['importance_mean']:.6f} | {row['importance_std']:.6f} |\n"

report += f"""

---

## 3. カテゴリ別分析

| Category | Total Importance | Avg Importance | N Features |
|----------|------------------|----------------|------------|
"""

for cat, row in category_stats.iterrows():
    report += f"| {cat.capitalize()} | {row['total_importance']:.6f} | {row['avg_importance']:.6f} | {int(row['n_features'])} |\n"

report += f"""

---

## 4. 主要な発見

### 4.1 カテゴリ別の重要度ランキング

1. **{category_stats.index[0].capitalize()}**: 合計重要度 {category_stats.iloc[0]['total_importance']:.4f}
2. **{category_stats.index[1].capitalize()}**: 合計重要度 {category_stats.iloc[1]['total_importance']:.4f}
3. **{category_stats.index[2].capitalize()}**: 合計重要度 {category_stats.iloc[2]['total_importance']:.4f}

### 4.2 最も重要な単一特徴量

**{importance_df.iloc[0]['feature']}** ({importance_df.iloc[0]['category']}カテゴリ)
- 重要度: {importance_df.iloc[0]['importance_mean']:.6f}
- この特徴量をシャッフルするとMAEが約 {importance_df.iloc[0]['importance_mean']:.2f}秒増加

"""

# レポート保存
report_path = os.path.join(output_dir, f'feature_importance_report_{timestamp}.md')
with open(report_path, 'w', encoding='utf-8') as f:
    f.write(report)

print(f"\nReport saved to: {report_path}")
print("\n" + "="*80)
print(report)
print("="*80)